## Arxiv--Research
## Tools creation
## wolframealpha -- math problem

In [40]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["WOLFRAM_ALPHA_APPID"] = os.getenv("WOLFRAM_ALPHA_APPID")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [32]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.tools import WolframAlphaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults

In [28]:
import asyncio
from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
import functools

wolfram = WolframAlphaAPIWrapper()

async def async_run(query):
    # Wrap the synchronous run method in an asynchronous function
    func = functools.partial(wolfram.run, query) # Pass the query
    result = await asyncio.to_thread(func)
    return result

try:
    result = asyncio.get_event_loop().run_until_complete(async_run("What is 2x+5 = -3x + 7?"))
    print(result)
except Exception as e:
    print(f"An error occurred: {e}")

Assumption: 2 x + 5 = -3 x + 7 
Answer: x = 2/5


## Tavily Search Tools

In [33]:
search = TavilySearchResults(max_results=3)
search.invoke("Who is donal tramp")

[{'url': 'https://en.wikipedia.org/wiki/Donald_Trump',
  'content': 'Donald Trump Donald Trump Donald Trump Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman serving as the 47th president of the United States since January 2025. Trumped!. Donald Trump By Donald Trump Donald Trump'},
 {'url': 'https://trumpwhitehouse.archives.gov/people/donald-j-trump/',
  'content': 'Trump – The White House Trump Trump Trump is the 45th President of the United States. Mr. Trump announced his candidacy for the Presidency on June 16, 2015. On November 8, 2016, Mr. Trump was elected President in the largest Electoral College landslide for a Republican in 28 years. Mr. Trump won more than 2,600 counties nationwide, the most since President Ronald Reagan in 1984. President Trump has delivered historic results in his first term in office despite partisan gridlock in the Nation’s Capital, and resistance from special interests and the Washington Establishment.

In [36]:
from langchain_community.tools import TavilySearchResults
tabli_search = TavilySearchResults(max_results=3)
tabli_search.invoke("Why is Shekh hachina is a bad PM")

[{'url': 'https://www.bbc.com/news/articles/cdrx31406g4o',
  'content': '"It\'s as if there\'s been some terrible tornado for 16 years and we\'re trying to pick up the pieces." Sheikh Hasina was elected prime minister in'},
 {'url': 'https://www.aljazeera.com/news/2024/8/5/sheikh-hasina-a-critical-misstep-and-the-end-of-15-years-of-rule',
  'content': "Throughout her extended tenure, Hasina's administration has been characterised by widespread political arrests and severe abuses, such as forced"},
 {'url': 'https://theconversation.com/bangladesh-how-the-prime-minister-sheikh-hasina-lost-her-grip-on-power-236170',
  'content': '... worst violence Bangladesh has ever seen. Within hours the prime minister, Sheikh Hasina, had resigned and fled the country in a helicopter'}]

## create wikipedia tools

In [34]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

## arxive tools

In [37]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [38]:
tools = [tabli_search, wiki, arxiv]

## Create Custom Tools [RAG-tools]

In [39]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [42]:
embeddings = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [44]:
loader = WebBaseLoader("https://github.com/MDalamin5/FGFI-Object-Detection-Using-Knowledge-Distillation")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=100
).split_documents(docs)

In [46]:
len(documents)

36

In [49]:
documents[int(len(documents)/2)]

Document(metadata={'source': 'https://github.com/MDalamin5/FGFI-Object-Detection-Using-Knowledge-Distillation', 'title': 'GitHub - MDalamin5/FGFI-Object-Detection-Using-Knowledge-Distillation: An efficient object detection project leveraging fine-grained feature imitation and knowledge distillation. This approach trains a lightweight student model (YOLOv5n) to outperform a larger teacher model (YOLOv5x) with fewer parameters, achieving state-of-the-art performance. Optimized for deployment on resource-constrained devices.', 'description': 'An efficient object detection project leveraging fine-grained feature imitation and knowledge distillation. This approach trains a lightweight student model (YOLOv5n) to outperform a larger teacher model (YOLOv5x) with fewer parameters, achieving state-of-the-art performance. Optimized for deployment on resource-constrained devices. - MDalamin5/FGFI-Object-Detection-Using-Knowledge-Distillation', 'language': 'en'}, page_content='mainBranchesTagsGo to

In [50]:
vectordb = FAISS.from_documents(documents=documents, embedding=embeddings)
retriever = vectordb.as_retriever()

c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [51]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000243592C8AA0>, search_kwargs={})

## Now make this retriever tools

In [52]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "kd-project-info", "Search any info about the project is knowledge distillation on object detection task")
retriever_tool.name

'kd-project-info'

In [53]:
tools = [wiki, arxiv, retriever_tool]

In [54]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Lenovo\\anaconda3\\envs\\torchGPU\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=2, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='kd-project-info', description='Search any info about the project is knowledge distillation on object detection task', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000024366843600>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_commun

In [60]:
tools[2].description

'Search any info about the project is knowledge distillation on object detection task'

## Agent Executer

In [77]:
## my LLM Model

groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model_name="qwen-2.5-32b", temperature=0.5, groq_api_key=groq_api_key)

## Now create a prompt template but using Langchain Prompt HUB

In [96]:
llm = ChatGroq(groq_api_key=groq_api_key, model_name="gemma2-9b-it")
tools = [wiki, arxiv, retriever_tool]

from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

## Creating Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [97]:
## agent executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [98]:
agent_executor.invoke({"input": "Tell me about generative ai and why i need to learn it?"})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Generative AI'}`


Page: Generative artificial intelligence
Summary: Generative artificial intelligence (generative AI, GenAI, or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These modeGenerative AI is a type of artificial intelligence that can create new content, such as text, images, audio, and video. 

There are several reasons why you might want to learn about generative AI:

* **Creativity and Innovation:** Generative AI can be a powerful tool for creative expression and innovation. It can help you generate new ideas, explore different concepts, and create unique content.
* **Automation:** Generative AI can automate many tasks that are currently done by humans, such as writing reports, generating code, and creating marketing materials. This can free up your time to focus on more strategic and creative work.
* **Personal

{'input': 'Tell me about generative ai and why i need to learn it?',
 'output': "Generative AI is a type of artificial intelligence that can create new content, such as text, images, audio, and video. \n\nThere are several reasons why you might want to learn about generative AI:\n\n* **Creativity and Innovation:** Generative AI can be a powerful tool for creative expression and innovation. It can help you generate new ideas, explore different concepts, and create unique content.\n* **Automation:** Generative AI can automate many tasks that are currently done by humans, such as writing reports, generating code, and creating marketing materials. This can free up your time to focus on more strategic and creative work.\n* **Personalization:** Generative AI can be used to create personalized experiences for users. For example, it can be used to generate personalized recommendations, create custom content, and tailor marketing messages.\n* **Research and Development:** Generative AI is a rap

In [100]:
agent_executor.invoke({"input": "Tell me about knowledge distillations"})




> Entering new AgentExecutor chain...

Invoking: `kd-project-info` with `{'query': 'knowledge distillation'}`


www.linkedin.com/in/mdalamin5/


Topics



  computer-vision


  imitation-learning


  knowledge-distillation


  yolov5



Resources





        Readme
 
License





     MIT license
    







Activity 
Stars




0
      stars 
Watchers




1
      watching 
Forks




0
      forks 


          Report repository
 






Releases
No releases published





Packages
      0

BCCD
0.873
0.881
0.916
0.628


Lemon Disease
0.928
0.888
0.925
0.693




Key Files

model.py: Implementation of YOLOv5x and YOLOv5n models.
loss.py: Definition of the Combined Imitation Loss Function (CILF).
train.py: Training pipeline for knowledge distillation.
datasets/: Scripts for loading Pascal VOC, BCCD, and other datasets.


Usage
Installation

Overview
This project demonstrates an approach to object detection that combines Knowledge Distillation with a novel Fine-Grained Feature Imitation m

{'input': 'Tell me about knowledge distillations',
 'output': 'This project focuses on object detection using a technique called knowledge distillation.  \n\nEssentially, they train a smaller, faster model (the "student") to imitate a larger, more accurate model (the "teacher"). This helps the smaller model learn to perform well even with fewer resources. \n\nThe project uses YOLOv5 as the base model, with YOLOv5x as the teacher and YOLOv5n as the student. They also incorporate a method called "Fine-Grained Feature Imitation" to further improve the student model\'s performance. \n\n\nLet me know if you have any more questions! \n'}